# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key and api_key[:8] == "sk-proj-":
    print("API Key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = "gpt-4o-mini"
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

In [5]:
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "linkedin page": "url": "https://www.linkedin.com/in/eddonner/"},
        {"type": "post page": "url": "https://edwarddonner.com/posts/"}
    ]
}
"""


In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.links} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
        Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of ['https://edwarddonner.com/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/', 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/'

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages=[
            {"role":"system", "content": link_system_prompt},
            {"role":"user", "content":get_links_user_prompt(website)}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'outsmart page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'posts page', 'url': 'https://edwarddonner.com/posts/'}]}

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-V3',
 '/deepseek-ai/DeepSeek-V3-Base',
 '/PowerInfer/SmallThinker-3B-Preview',
 '/black-forest-labs/FLUX.1-dev',
 '/hexgrad/Kokoro-82M',
 '/models',
 '/spaces/osanseviero/gemini-coder',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/akhaliq/anychat',
 '/spaces',
 '/datasets/agibot-world/AgiBotWorld-Alpha',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/PowerInfer/QWQ-LONGCOT-500K',
 '/datasets/cfahlgren1/react-code-instructions',
 '/datasets/HuggingFaceTB/finemath',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/tra

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/brand'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page: \n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found Links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page: 
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepseek-ai/DeepSeek-V3
Updated
6 days ago
•
62.8k
•
1.16k
deepseek-ai/DeepSeek-V3-Base
Updated
6 days ago
•
7.93k
•
1.14k
PowerInfer/SmallThink

In [28]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [30]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page: \nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-V3\nUpdated\n6 days ago\n•\n62.8k\n•\n1.16k\ndeepseek-ai/DeepSeek-V3-Base\nUpdated\n6 days ago\n•\n7.93k\n•\n1.14k\nPowerInfer/SmallThinker-3B-Preview\nUpdated\n1 day ago\n•\n2.93k\n•\n216\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16, 2024\n•\n1.15M\n•\n7.72k\nhexgrad/Kokoro-82M\nUpdated\nabout 1 hour ago\n•\n717\n•\n195\nBrowse 400k+ models\nSpaces\nRunning\n456\n🦀\nGemini Coder\nRunning\non\nZero\n2.53k\n🏢\

In [31]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [32]:
create_brochure("HuggingFace", "https://huggingface.com")

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


```markdown
# 🌟 Hugging Face: The AI Community Building the Future! 🌟

Hey there, future AI enthusiasts, investors, and apex predators of talent acquisition! Welcome to the wild and wonderful world of Hugging Face, where code meets creativity, and logic gives hugs like it's never been hugged before. 

## 🤖 What is Hugging Face?

Think of us as **the playground** for machine learning aficionados. We host a thriving community where clever minds collaborate on models, datasets, and applications with the enthusiasm of a kid in a candy store. With over **400k models and 100k datasets**, we might as well be your new favorite source of digital confectionery!

### Current Trends (Not Just Your Fun Uncle's Fashion)
- **Models:** From deep-seekers to brainy previews, we’ve got a model for every mood. Check out our trending models like *DeepSeek-V3* and *FLUX.1-dev*—because who doesn't love giving their AI a little personality?
  
## 💻 Why Hugging Face?

- **Your Future in the Cloud:** Whether you’ve got a penchant for pretty pixels or an obsession with optimal algorithms, you can, quite literally, **run with it**. We have **456 Spaces** where your AI can work, learn, and, maybe someday, get a medal for participation in the Olympics of Tech.
  
- **Open Source Love:** Join our community that’s all about sharing! We provide state-of-the-art tools for Pytorch, TensorFlow, and even JAX because we love playing with all kinds of tech toys.
  
- **Affordable AI:** Start your journey in the cloud starting at just **$0.60 hour for GPU**! Yes, you read that right! Your AI dreams may not be the most expensive date you've ever had... at least not on our watch!

## 🎉 Who Are Our Customers?

From **AI at Meta** to **Amazon Web Services** and even **Google**, we have over **50,000 organizations** that trust us enough to build their AI infrastructures. That's right, we’re basically the coolest club in town – and we have a no-judgment policy on cake consumption during data analysis sessions.

## 🌈 Culture Club: Join Us!

What’s life at Hugging Face like? Imagine a place where challenges are seen as exciting puzzles, and every ‘bug’ is just a fancy way of saying “opportunity for improvement." Here, brainstorming sessions often involve pizza and an excessive amount of caffeinated beverages. 

### Career Opportunities:
In essence, we’re serious about being fun! We’re always on the lookout for enthusiastic recruits (after all, we need people who can handle an AI that’s learned to maximize fun as well as efficiency). 

### Perks of Working with Us:
- A free hug (virtually, of course).
- Access to all the latest tech goodies.
- Daily doses of motivation from community vibes.
- The chance to contribute to the future of AI, one model at a time.

## ✨ Join the Hugging Face Family!

Come for the AI, stay for the community hugs! Are you ready to embrace the future? Sign up today and start building your AI dreams with us, one (overly complicated) model at a time!

**Contact Us:** [Sign Up](#) | [Join Our Discord](#) | [Follow on GitHub](#)

---

Hugging Face: Because who wouldn't want to cuddle with their AI?
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://discuss.huggingface.co'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}



# Hugging Face Brochure

## The AI Community Building the Future

**Welcome to Hugging Face**, the leading platform where the machine learning community collaborates on models, datasets, and applications. Our vision is to empower everyone to contribute to the AI landscape, making advanced machine learning technologies accessible and beneficial for all.

---

### Our Offerings

- **Models**: Browse and utilize over 400,000 machine learning models to accelerate your AI projects.
- **Datasets**: Access a vast collection of over 100,000 curated datasets tailored for diverse applications.
- **Spaces**: Create and share applications that leverage the power of machine learning in a collaborative environment.

---

### Built for Collaboration

At Hugging Face, collaboration is at the heart of what we do. We encourage users to share their work and build their Machine Learning profiles by hosting and collaborating on unlimited public models, datasets, and applications. Join thousands who are already part of our thriving ecosystem.

---

### Enterprise Solutions

For organizations seeking advanced security and dedicated support, our **Enterprise** offerings include:

- **Optimized Compute**: Get started with GPU deployments at competitive pricing, starting at $0.60/hour.
- **Scalable Options**: Enhance your team’s productivity with features like Single Sign-On, priority support, and audit logs.

More than 50,000 organizations, including industry giants like Meta, Google, and Microsoft, trust Hugging Face for their machine learning needs.

---

### Our Culture

At Hugging Face, we pride ourselves on cultivating an inclusive and open company culture. We believe in the power of teamwork and innovation, where every member contributes to building state-of-the-art machine learning tools. We support professional growth and advocacy for diverse perspectives in the tech community.

---

### Join Us

Are you passionate about AI and machine learning? We're always looking for talent across various domains. Explore career opportunities with us and help shape the future of AI. We value creativity, collaboration, and a commitment to excellence.

- **Explore Careers**: [Hugging Face Jobs](https://huggingface.co/jobs)

---

### Connect With Us

Stay updated with the latest trends and community insights:

- [Twitter](https://twitter.com/huggingface)
- [GitHub](https://github.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)

**Join the AI revolution with Hugging Face – together, we’re building the future.**

---

*For additional information, visit our [Website](https://huggingface.co).*




In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
stream_brochure("HuggingFace", "https://huggingface.co")

Found Links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
### The AI Community Building the Future

Hugging Face is an innovative platform where the machine learning community collaborates on models, datasets, and applications. With a focus on accessibility, we empower both newcomers and seasoned professionals to create, discover, and enhance machine learning workflows collectively.

---

## Our Offerings

- **Models**: Access and collaborate on a vast library of over 400,000 models, from text generation to image processing.
- **Datasets**: Share and discover a diverse range of over 100,000 datasets tailored for various ML tasks.
- **Spaces**: Explore a range of applications designed to showcase what's possible in machine learning, with 150,000+ applications currently available.
- **Enterprise Solutions**: Leverage advanced machine learning capabilities with enterprise-grade security and dedicated support.

### Trending Models
- `deepseek-ai/DeepSeek-V3` (62.8k updates)
- `PowerInfer/SmallThinker-3B-Preview` (2.93k updates)
- `black-forest-labs/FLUX.1-dev` (1.15M updates)

---

## Our Customers
More than **50,000 organizations** trust Hugging Face, including:
- **Meta**: Over 2,000 models developed
- **Amazon Web Services**: 21 active models
- **Google**: 913 models utilized
- **Intel** and **Microsoft**: Active participants with numerous models contributing to the ecosystem.

---

## Company Culture
At Hugging Face, we foster a collaborative and open-source environment, where creativity and innovation thrive. Our culture is built on the principles of community engagement, transparency, and shared learning. We believe that by working together, we can build better solutions and make AI more accessible to everyone.

---

## Careers at Hugging Face
Join us in shaping the future of AI! We are always on the lookout for talent who are passionate about machine learning and eager to contribute to a vibrant community. 

- **Open Positions**: Explore exciting job opportunities across various roles including engineering, product management, and community engagement.
- **Growth Opportunities**: Embrace a culture of continuous learning and development.

---

## Join the Hugging Face Community
- **Website**: [huggingface.co](https://huggingface.co)
- **Follow us**: 
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://linkedin.com/company/huggingface)
  - [Discord](https://discord.gg/huggingface)

Explore the future of AI with Hugging Face today! Whether you are a customer, investor, or prospective team member, we invite you to join our community and help us shape what's next in machine learning.

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    for chunk in stream:
        print(chunk.choices[0].delta.content or '', end='')
    
    # response = ""
    # display_handle = display(Markdown(""), display_id=True)
    # for chunk in stream:
    #     response += chunk.choices[0].delta.content or ''
    #     response = response.replace("```","").replace("markdown", "")
    #     update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
stream_brochure("HuggingFace", "https://huggingface.co")

Found Links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/brand'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
# Hugging Face Brochure

## About Hugging Face
Hugging Face is revolutionizing the machine learning landscape as a collaborative platform focused on building the future of AI. With a vibrant community of developers and researchers, we provide the tools and infrastructure needed to create, share, and improve machine learning models, datasets, and applications.

## Our Offerings
- **Models**: Access a rich library of over 400,000 

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>